In [8]:
import tensorflow as tf
import tf_keras
import tensorflow_model_optimization as tfmot
from tf_keras import layers,losses,optimizers,Sequential
from tf_keras.models import Model
from tf_keras.layers import Dense, Flatten, Dropout, GlobalAveragePooling2D,Input
from tf_keras.applications import VGG16
from tf_keras import datasets,models
import importlib

In [9]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [10]:
from importing_dataset import load_dataset
import Entropy
importlib.reload(Entropy)
from APoZ import APoZ_Algorithm
train_examples, validation_examples, num_examples, num_classes, class_names = load_dataset('horses_or_humans', 70)

In [11]:
from reformatting import reformat_image
BATCH_SIZE = 32
IMG_SIZE = (224, 224)
train_batches = train_examples.cache().shuffle(num_examples//4).map(reformat_image).batch(BATCH_SIZE).prefetch(1)
validation_batches = validation_examples.map(reformat_image).batch(BATCH_SIZE).prefetch(1)
train_batches

<_PrefetchDataset element_spec=(TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None,), dtype=tf.int64, name=None))>

In [45]:
# Initialiser les couches une par une
inputs = Input(shape=(224, 224, 3))

# Bloc 1
x = layers.Conv2D(64, (3, 3), activation='relu', padding='same', name='block1_conv1')(inputs)
x = layers.Conv2D(64, (3, 3), activation='relu', padding='same', name='block1_conv2')(x)
x = layers.MaxPooling2D((2, 2), strides=(2, 2), name='block1_pool')(x)

# Bloc 2
x = layers.Conv2D(128, (3, 3), activation='relu', padding='same', name='block2_conv1')(x)
x = layers.Conv2D(128, (3, 3), activation='relu', padding='same', name='block2_conv2')(x)
x = layers.MaxPooling2D((2, 2), strides=(2, 2), name='block2_pool')(x)

# Bloc 3
x = layers.Conv2D(256, (3, 3), activation='relu', padding='same', name='block3_conv1')(x)
x = layers.Conv2D(256, (3, 3), activation='relu', padding='same', name='block3_conv2')(x)
x = layers.Conv2D(256, (3, 3), activation='relu', padding='same', name='block3_conv3')(x)
x = layers.MaxPooling2D((2, 2), strides=(2, 2), name='block3_pool')(x)

# Bloc 4
x = layers.Conv2D(512, (3, 3), activation='relu', padding='same', name='block4_conv1')(x)
x = layers.Conv2D(512, (3, 3), activation='relu', padding='same', name='block4_conv2')(x)
x = layers.Conv2D(512, (3, 3), activation='relu', padding='same', name='block4_conv3')(x)
x = layers.MaxPooling2D((2, 2), strides=(2, 2), name='block4_pool')(x)

# Bloc 5
x = layers.Conv2D(512, (3, 3), activation='relu', padding='same', name='block5_conv1')(x)
x = layers.Conv2D(512, (3, 3), activation='relu', padding='same', name='block5_conv2')(x)
x = layers.Conv2D(512, (3, 3), activation='relu', padding='same', name='block5_conv3')(x)
x = layers.MaxPooling2D((2, 2), strides=(2, 2), name='block5_pool')(x)

# Ajouter des couches supplémentaires
x = layers.GlobalMaxPooling2D(name="global_max_pool")(x)
x = layers.Dropout(0.3, name="dropout")(x)
outputs = layers.Dense(2, activation="softmax", name="predictions")(x)

# Construire le modèle
model = Model(inputs, outputs, name="custom_vgg16.keras")

# Charger les poids de VGG16
vgg16_base = VGG16(include_top=False, weights="imagenet", input_shape=(224, 224, 3))

# Appliquer les poids pour chaque couche manuellement
for layer, pretrained_layer in zip(model.layers, vgg16_base.layers):
    if isinstance(layer, layers.Conv2D):
        layer.set_weights(pretrained_layer.get_weights())


In [184]:
# Charger votre modèle
importlib.reload(Entropy)
from Entropy import EntropyPruning
testt_model = models.load_model("model_experiments/custom_vgg16.keras")
testt_model.summary()

Model: "custom_vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_40 (InputLayer)       [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)      

In [202]:
pruning = EntropyPruning(model=testt_model, threshold=0.03)
pruned_model = pruning.prune_model(None)
pruned_model.summary()



Model: "sequential_76"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  multiple                  0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  multiple                  0         
                                                                 
 block3_conv1 (Conv2D)       (None, 56, 56, 256)     

In [192]:
from tf_keras import optimizers
import tempfile
import tensorflow_model_optimization as tfmot
# Finish pruning after 2 epochs
epochs = 2
BATCH_SIZE = 32

def prune_model(model, initial_sparsity, final_sparsity, train_data=train_batches, val_data=validation_batches, epochs=epochs, feature_extractor=model ):

  # Create a tensorboard logfile
  logdir = tempfile.mkdtemp()
  # The end_step is the total number of iterations required for the training data which is basically the entire epochs over the length of the training data
  end_step = int(len(train_data) * epochs * 0.5)
  # Import the low-magnitude-pruning function
  prune_low_magnitude = tfmot.sparsity.keras.prune_low_magnitude

  # Set the prunung params
  pruning_params = {

      "pruning_schedule" : tfmot.sparsity.keras.PolynomialDecay(initial_sparsity=0,
                                                                final_sparsity=final_sparsity,
                                                                begin_step=0,
                                                                end_step=end_step)

                  }


  learning_rate_fn = optimizers.schedules.PolynomialDecay(
    0.001,
    1000,
    0.0001,
    power=0.5)

  # Model for pruning
  #feature_extractor = prune_low_magnitude(feature_extractor, **pruning_params)
  model_for_pruning = prune_low_magnitude(model, **pruning_params)

  # Recompile
  model_for_pruning.compile(optimizer= optimizers.Adam(),
                            loss= losses.SparseCategoricalCrossentropy(),
                            metrics=["accuracy"])
  #create callbacks
  callbacks = [tfmot.sparsity.keras.UpdatePruningStep(),
              tfmot.sparsity.keras.PruningSummaries(log_dir=logdir),
               #create_model_checkpoint(model_name=model.name),
              #tf.keras.callbacks.ReduceLROnPlateau(monitor="val_loss",
                                                  #patience=3,
                                                  #verbose=1),
               #tf.keras.callbacks.EarlyStopping(monitor="val_loss",
                                                        #patience=4,
                                                        #restore_best_weights=True)
                                                        ]

  # Fit the model
  model_for_pruning.fit(train_data,
                      validation_data=val_data,
                      batch_size=BATCH_SIZE,
                      epochs=epochs,
                      callbacks=callbacks)

  # Save the model
  #model_for_pruning.save(f"mnist_model_sparsity_{final_sparsity}")

  # Evaluate the model
  score = model_for_pruning.evaluate(val_data, verbose=0)
  metric_dict = {
      "sparsity" : final_sparsity,
      "val_loss" : np.round(score[0], 4),
      "val_accuracy" : np.round(score[1] * 100, 4)
  }
  return logdir, metric_dict, model_for_pruning

In [203]:
# Choose the best sparsity level
final_sparsity = 0.80

logdir, metrics, pruned_model = prune_model(model=pruned_model,
          initial_sparsity=0,
          final_sparsity=final_sparsity,
          epochs=3)

Epoch 1/3
23/23 [==============================] - 159s 7s/step - loss: 0.6922 - accuracy: 0.5257 - val_loss: 0.6947 - val_accuracy: 0.4838
Epoch 2/3
23/23 [==============================] - 174s 8s/step - loss: 0.6920 - accuracy: 0.5257 - val_loss: 0.6949 - val_accuracy: 0.4838
Epoch 3/3
23/23 [==============================] - 145s 6s/step - loss: 0.6920 - accuracy: 0.5257 - val_loss: 0.6949 - val_accuracy: 0.4838


In [196]:
model_for_export = tfmot.sparsity.keras.strip_pruning(pruned_model)

_, pruned_keras_file = tempfile.mkstemp('.h5')
models.save_model(model_for_export, pruned_keras_file, include_optimizer=False)
print('Saved pruned Keras model to:', pruned_keras_file)

C:\Users\neyen\AppData\Local\Temp\ipykernel_71452\369930696.py:4: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native TF-Keras format, e.g. `model.save('my_model.keras')`.
  models.save_model(model_for_export, pruned_keras_file, include_optimizer=False)


Saved pruned Keras model to: C:\Users\neyen\AppData\Local\Temp\tmpjh1lij65.h5
